# Test

https://stackoverflow.com/questions/51700729/how-to-construct-a-network-with-two-inputs-in-pytorch

https://stackoverflow.com/questions/46058190/passing-multiple-inputs-to-cnn-model

https://pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

## pytorch 串接方法

In [ ]:
import torch
from torch import nn

class TwoInputsNet(nn.Module):
    def __init__(self):
        super(TwoInputsNet, self).__init__()
        self.conv = nn.Conv2d( ... )  # set up your layer here
        self.fc1 = nn.Linear( ... )  # set up first FC layer
        self.fc2 = nn.Linear( ... )  # set up the other FC layer

    def forward(self, input1, input2):
        c = self.conv(input1)
        f = self.fc1(input2)
        # now we can reshape `c` and `f` to 2D and concat them
        combined = torch.cat((c.view(c.size(0), -1),
                              f.view(f.size(0), -1)), dim=1)
        out = self.fc2(combined)
        return out

## keras 串接方法

In [ ]:
Arthurfrom keras.layers import *

input1 = Input((75,))
input2 = Input((1440,))

name = Embedding(max_features, 8, input_length=maxlen)(input1)
mails = Reshape((1440,1))(input2) #adding 1 channel at the end   

#
name = Conv1D( feel free to customize )(name)
name = Conv1D( feel free to customize )(name)

mails = Conv1D( feel free to customize )(mails)
mails = Conv1D( feel free to customize )(mails)

#
name = Flatten()(name)
mails = Flatten()(mails)
out = Concatenate()([name,mails])

out = add your extra layers

out = Dense(2,activation='softmax')(out)

#
from keras.models import Model
model = Model([input1,input2], out)

#
model.fit([xName,xMails], Y, ....)

## keras CNN範例

In [ ]:
def CNN(clusters=2):
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 100), activation='relu', input_shape=(30, 100, 1)))
    model.add(layers.MaxPooling2D((28, 1))) # 30-3+1 = 28 !!!
    model.add(layers.Dropout(0.4))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(clusters, activation='softmax'))
    model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
    return model

## EB CNN

In [1]:
import torch
from torch import nn
import numpy as np
import torch.optim as optim

# X = X.values.reshape(X.shape[0], 30, 100, 1).astype('float32')
# n*30*100*1 => n*1*100
# X_test = X_test.values.reshape(X_test.shape[0], 30, 100, 1).astype('float32')
# n*30*100*1 => n*1*100

# (64, (3, 100), (30, 100, 1))
# (64, 1, (3, 100), (1, 100))

def to_category(num):
#     if num >= 0.5:
    if num >= 0:
        return torch.tensor(1)
    else:
        return torch.tensor(0)
#     num[num >= 0] = 1
#     num[num < 0] = 0

class ThreeInputsCNN(nn.Module):
    def __init__(self):
        super(ThreeInputsCNN, self).__init__()
        self.conv1 = nn.Sequential(
          nn.Conv2d(1, 100, (3, 100), stride=(1, 1)),
          nn.MaxPool2d((28, 1)),
          nn.Flatten()
        )
        self.conv2 = nn.Sequential(
          nn.Conv2d(1, 100, (3, 100), stride=(1, 1)),
          nn.MaxPool2d((5, 1)),
          nn.Flatten()
        )
        self.fc = nn.Sequential(
          nn.Linear(3, 1),
          nn.Flatten(0),
          nn.Sigmoid(),
          nn.Linear(100, 2),
          nn.Sigmoid()
        )
#         self.fc1 = nn.Linear( ... )  # set up first FC layer


    def forward(self, input1, input2, input3):
        c1 = self.conv1(input1)
        c2 = self.conv2(input2)
        c3 = input3
#         print(c1.shape, c2.shape, c3.shape)
#         print(c1, c2, c3)
        # now we can reshape `c` and `f` to 2D and concat them
        combined = torch.cat( 
                            #
                            (    c1.view(c1.size(0), -1)
                               , c2.view(c2.size(0), -1)
                               , c3.view(c3.size(0), -1)
                            )
                            #
                                        , dim=1 )
#         print(combined.shape)
        out = self.fc(combined)
        return out

/opt/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 以1家公司30天為假設

input1 = torch.randn(30, 30, 100) # 30*monthly news
input2 = torch.randn(30, 7, 100) # 30*weekly news
input3 = torch.randn(100, 30) # daily news*30
# input4 = torch.randn(100, 1) # company
input5 = torch.randn(30) # outcome

# model = ThreeInputsCNN.forward(input1, input2, input3)

In [3]:
model = ThreeInputsCNN()

In [4]:
input1.shape

torch.Size([30, 30, 100])

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(100):
    
    running_loss = 0.0
    
    
    for i in range(input1.shape[0]):
        n = input1.shape[0]
        input_1 = torch.unsqueeze(input1[i, :, :], 0)
        input_2 = torch.unsqueeze(input2[i, :, :], 0)
        input_3 = torch.unsqueeze(input3[:, i], 1)

    #         output = torch.unsqueeze( to_category(model.forward(input_1, input_2, input_3)), 0 )
        label = torch.unsqueeze( to_category(input5[i]), 0)
#         label = to_category(input5)
        output = torch.unsqueeze( model.forward(input_1, input_2, input_3), 0 )
    #         label = torch.unsqueeze( input5[i], 0)
        print(epoch, output, label)
        loss = criterion(output, label)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

0 tensor([[0.3592, 0.3828]], grad_fn=<UnsqueezeBackward0>) tensor([0])
0 tensor([[0.3725, 0.3619]], grad_fn=<UnsqueezeBackward0>) tensor([1])
0 tensor([[0.3652, 0.3797]], grad_fn=<UnsqueezeBackward0>) tensor([0])
0 tensor([[0.3843, 0.3345]], grad_fn=<UnsqueezeBackward0>) tensor([0])
0 tensor([[0.3835, 0.3548]], grad_fn=<UnsqueezeBackward0>) tensor([0])
0 tensor([[0.3654, 0.3649]], grad_fn=<UnsqueezeBackward0>) tensor([0])
0 tensor([[0.3789, 0.3338]], grad_fn=<UnsqueezeBackward0>) tensor([0])
0 tensor([[0.4098, 0.3662]], grad_fn=<UnsqueezeBackward0>) tensor([0])
0 tensor([[0.3952, 0.3315]], grad_fn=<UnsqueezeBackward0>) tensor([1])
0 tensor([[0.4289, 0.3257]], grad_fn=<UnsqueezeBackward0>) tensor([0])
0 tensor([[0.4507, 0.3140]], grad_fn=<UnsqueezeBackward0>) tensor([1])
0 tensor([[0.4415, 0.3015]], grad_fn=<UnsqueezeBackward0>) tensor([1])
0 tensor([[0.4724, 0.2794]], grad_fn=<UnsqueezeBackward0>) tensor([1])
0 tensor([[0.4942, 0.2472]], grad_fn=<UnsqueezeBackward0>) tensor([0])
0 tens

5 tensor([[0.1450, 0.6136]], grad_fn=<UnsqueezeBackward0>) tensor([1])
5 tensor([[0.0538, 0.7622]], grad_fn=<UnsqueezeBackward0>) tensor([1])
5 tensor([[0.6599, 0.1335]], grad_fn=<UnsqueezeBackward0>) tensor([0])
5 tensor([[0.6589, 0.1465]], grad_fn=<UnsqueezeBackward0>) tensor([0])
5 tensor([[0.6887, 0.0933]], grad_fn=<UnsqueezeBackward0>) tensor([0])
5 tensor([[0.1190, 0.6544]], grad_fn=<UnsqueezeBackward0>) tensor([1])
5 tensor([[0.7161, 0.1113]], grad_fn=<UnsqueezeBackward0>) tensor([0])
5 tensor([[0.7580, 0.0664]], grad_fn=<UnsqueezeBackward0>) tensor([0])
5 tensor([[0.0112, 0.8884]], grad_fn=<UnsqueezeBackward0>) tensor([1])
5 tensor([[0.7349, 0.0995]], grad_fn=<UnsqueezeBackward0>) tensor([0])
5 tensor([[0.0673, 0.7166]], grad_fn=<UnsqueezeBackward0>) tensor([1])
5 tensor([[0.0400, 0.8172]], grad_fn=<UnsqueezeBackward0>) tensor([1])
5 tensor([[0.0910, 0.7295]], grad_fn=<UnsqueezeBackward0>) tensor([1])
5 tensor([[0.7696, 0.0903]], grad_fn=<UnsqueezeBackward0>) tensor([0])
5 tens

11 tensor([[1.0000e+00, 8.9822e-12]], grad_fn=<UnsqueezeBackward0>) tensor([0])
11 tensor([[2.2550e-09, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
11 tensor([[1.0000e+00, 1.4176e-09]], grad_fn=<UnsqueezeBackward0>) tensor([0])
11 tensor([[1.0000e+00, 1.3735e-15]], grad_fn=<UnsqueezeBackward0>) tensor([0])
11 tensor([[1.0000e+00, 2.2228e-13]], grad_fn=<UnsqueezeBackward0>) tensor([0])
11 tensor([[1.0000e+00, 3.0350e-11]], grad_fn=<UnsqueezeBackward0>) tensor([0])
11 tensor([[1.0000e+00, 9.6502e-13]], grad_fn=<UnsqueezeBackward0>) tensor([0])
11 tensor([[1.0000e+00, 3.2771e-12]], grad_fn=<UnsqueezeBackward0>) tensor([0])
11 tensor([[1.0880e-09, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
11 tensor([[1.0000e+00, 4.3555e-14]], grad_fn=<UnsqueezeBackward0>) tensor([0])
11 tensor([[6.6124e-10, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
11 tensor([[6.6315e-14, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
11 tensor([[4.7969e-06, 9.9704e-01]], gr

16 tensor([[8.8878e-21, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
16 tensor([[3.2405e-08, 9.9999e-01]], grad_fn=<UnsqueezeBackward0>) tensor([1])
16 tensor([[1.0000e+00, 1.2835e-12]], grad_fn=<UnsqueezeBackward0>) tensor([0])
16 tensor([[7.0417e-13, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
16 tensor([[2.4056e-14, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
16 tensor([[3.9863e-05, 9.9765e-01]], grad_fn=<UnsqueezeBackward0>) tensor([1])
16 tensor([[1.0000e+00, 1.2553e-18]], grad_fn=<UnsqueezeBackward0>) tensor([0])
16 tensor([[1.0000e+00, 3.4830e-14]], grad_fn=<UnsqueezeBackward0>) tensor([0])
16 tensor([[1.0000e+00, 4.5633e-24]], grad_fn=<UnsqueezeBackward0>) tensor([0])
16 tensor([[1.0689e-12, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
16 tensor([[1.0000e+00, 1.8889e-18]], grad_fn=<UnsqueezeBackward0>) tensor([0])
16 tensor([[1.0000e+00, 1.8051e-19]], grad_fn=<UnsqueezeBackward0>) tensor([0])
16 tensor([[4.7050e-20, 1.0000e+00]], gr

21 tensor([[1.0000e+00, 3.6389e-10]], grad_fn=<UnsqueezeBackward0>) tensor([0])
21 tensor([[7.7199e-17, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
21 tensor([[9.7103e-18, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
21 tensor([[2.6878e-09, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
21 tensor([[1.0000e+00, 7.6126e-27]], grad_fn=<UnsqueezeBackward0>) tensor([0])
21 tensor([[1.0000e+00, 1.0333e-15]], grad_fn=<UnsqueezeBackward0>) tensor([0])
21 tensor([[1.0000e+00, 1.1026e-36]], grad_fn=<UnsqueezeBackward0>) tensor([0])
21 tensor([[3.1190e-14, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
21 tensor([[1.0000e+00, 5.6709e-26]], grad_fn=<UnsqueezeBackward0>) tensor([0])
21 tensor([[1.0000e+00, 4.5025e-21]], grad_fn=<UnsqueezeBackward0>) tensor([0])
21 tensor([[9.5298e-26, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
21 tensor([[1.0000e+00, 2.1207e-32]], grad_fn=<UnsqueezeBackward0>) tensor([0])
21 tensor([[2.4980e-21, 1.0000e+00]], gr

...

74 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
74 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
74 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
74 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
74 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
74 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
74 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
74 tensor([[9.5290e-28, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
74 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
74 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
74 tensor([[1.8874e-10, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
75 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
75 tensor([[3.7130e-23, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
75 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
75 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tens

79 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
79 tensor([[1.0000e+00, 4.6402e-08]], grad_fn=<UnsqueezeBackward0>) tensor([0])
79 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
79 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
79 tensor([[3.3516e-06, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
79 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
79 tensor([[1.0000e+00, 4.1669e-09]], grad_fn=<UnsqueezeBackward0>) tensor([0])
79 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
79 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
79 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
79 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
79 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
79 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
79 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
79 tensor([[3.2632e-22, 1.0000e+00]], grad_fn=<Unsqueeze

84 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
84 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
84 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
84 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
84 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
84 tensor([[1.3015e-23, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
84 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
84 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
84 tensor([[2.2726e-18, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
85 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
85 tensor([[1.3205e-28, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
85 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
85 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
85 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
85 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tens

90 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
90 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
90 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
90 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
90 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
90 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
90 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
90 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
90 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
90 tensor([[1.0000e+00, 7.1584e-09]], grad_fn=<UnsqueezeBackward0>) tensor([0])
90 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
90 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
90 tensor([[5.3343e-07, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
90 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
90 tensor([[1.0000e+00, 1.5119e-05]], grad_fn=<UnsqueezeBackward0>) tens

95 tensor([[2.5791e-18, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
96 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
96 tensor([[8.9686e-37, 1.0000e+00]], grad_fn=<UnsqueezeBackward0>) tensor([1])
96 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
96 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
96 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
96 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
96 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
96 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
96 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
96 tensor([[1., 0.]], grad_fn=<UnsqueezeBackward0>) tensor([0])
96 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
96 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
96 tensor([[0., 1.]], grad_fn=<UnsqueezeBackward0>) tensor([1])
96 tensor([[1.0000e+00, 1.2054e-10]], grad_fn=<UnsqueezeBackward0>) tens

## EB CNN (Batched Version)

In [65]:
import torch
from torch import nn
import numpy as np
import torch.optim as optim

# X = X.values.reshape(X.shape[0], 30, 100, 1).astype('float32')
# n*30*100*1 => n*1*100
# X_test = X_test.values.reshape(X_test.shape[0], 30, 100, 1).astype('float32')
# n*30*100*1 => n*1*100

# (64, (3, 100), (30, 100, 1))
# (64, 1, (3, 100), (1, 100))

def to_category(num):
#     if num >= 0.5:
#     if num >= 0:
#         return torch.tensor(1)
#     else:
#         return torch.tensor(0)
    num[num > 0] = 1
    num[num <= 0] = 0

class ThreeInputsCNN(nn.Module):
    def __init__(self):
        super(ThreeInputsCNN, self).__init__()
        self.conv1 = nn.Sequential(
          nn.Conv2d(1, 100, (3, 100), stride=(1, 1)),
          nn.MaxPool2d((28, 1)),
          nn.Flatten(),
          nn.Flatten()
        )
        self.conv2 = nn.Sequential(
          nn.Conv2d(1, 100, (3, 100), stride=(1, 1)),
          nn.MaxPool2d((5, 1)),
          nn.Flatten(),
          nn.Flatten()
        )
        self.fc = nn.Sequential(
          nn.Linear(3, 1),
          nn.Flatten(),
          nn.Sigmoid(),
          nn.Linear(100, 2),
          nn.Sigmoid()
#           nn.Flatten(0)
        )
#         self.fc1 = nn.Linear( ... )  # set up first FC layer


    def forward(self, input1, input2, input3):
        c1 = self.conv1(input1)
        c2 = self.conv2(input2)
#         print(c1.shape,c2.shape)
        c3 = input3
#         print(c1.shape, c2.shape, c3.shape)
#         print(c1, c2, c3)
        # now we can reshape `c` and `f` to 2D and concat them
    
        self.n = c1.size(0)
        combined = torch.empty(self.n, 100, 3)
        combined[:, :, 0] = c1
        combined[:, :, 1] = c2
        combined[:, :, 2] = c3
        
        
        
#         combined = torch.cat( 
#                             #
#                             (    c1.view(c1.size(0), -1)
#                                , c2.view(c2.size(0), -1)
#                                , c3.view(c3.size(0), -1)
#                             )
#                             #
#                                         , dim=1 )
#         print(combined.shape)
        out = self.fc(combined)
        return out

In [75]:
# 以1家公司30天為假設

input1 = torch.randn(30, 1, 30, 100) # 30*monthly news
input2 = torch.randn(30, 1, 7, 100) # 30*weekly news
input3 = torch.randn(30, 100) # daily news*30
# input4 = torch.randn(100, 1) # company
input5 = torch.randn(30) # outcome

# model = ThreeInputsCNN.forward(input1, input2, input3)

In [76]:
model = ThreeInputsCNN()

In [92]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

to_category(input5)
label = torch.from_numpy(input5.numpy().astype(int))

for epoch in range(100):
    
    running_loss = 0.0
    
    output = model.forward(input1, input2, input3)
#     print(epoch, output, label)
    loss = criterion(output, label)

    loss.backward()
    optimizer.step()

    print(loss.item())
    running_loss = 0.0

print('Finished Training')

0.47788897156715393
0.47788819670677185
0.47788694500923157
0.47788554430007935
0.47788432240486145
0.47788336873054504
0.4778827428817749
0.47788235545158386
0.47788211703300476
0.47788190841674805
0.4778817892074585
0.4778815805912018
0.477881520986557
0.47788140177726746
0.4778813421726227
0.4778812825679779
0.4778812825679779
0.4778812527656555
0.4778812825679779
0.4778813421726227
0.47788143157958984
0.4778815507888794
0.4778817892074585
0.4778820872306824
0.47788241505622864
0.47788286209106445
0.4778834879398346
0.4778841733932495
0.4778848886489868
0.4778856933116913
0.4778864085674286
0.47788700461387634
0.47788724303245544
0.4778871238231659
0.4778865575790405
0.4778856635093689
0.47788459062576294
0.4778834581375122
0.477882444858551
0.47788164019584656
0.4778810143470764
0.4778806269168854
0.4778803288936615
0.47788017988204956
0.47788006067276
0.47788000106811523
0.47787997126579285
0.47788000106811523
0.47788000106811523
0.47788000106811523
0.47788006067276
0.477880120277

In [366]:
from wakepy import set_keepawake, unset_keepawake
set_keepawake(keep_screen_awake=True)

# Start: label

### 2 monthly news -> 125 individual company(read 125 & stockReturns) -> feed in

In [1]:
import torch
from torch import nn
import os
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import re
import ast
import json
from sklearn.metrics import confusion_matrix
import warnings

class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.ln1 = nn.Bilinear(100, 100, 100)
        self.ln1_2 = nn.Linear(200, 100, bias=False)
        self.ln2 = nn.Bilinear(100, 100, 100)
        self.ln2_2 = nn.Linear(200, 100, bias=False)
        self.ln3 = nn.Bilinear(100, 100, 100)
        self.ln3_2 = nn.Linear(200, 100, bias=False)
        self.ln4 = nn.Linear(100, 1, bias=True)
#         self.ln0 = nn.functional.linear(weight, bias)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    def forward1(self, input1, input2):
        c1 = self.ln1(input1, input2)
        combined = c1
        return combined

    def forward2(self, input1, input2):
        c1 = self.ln2(input1, input2)
        combined = c1
        return combined

    def forward3(self, input1, input2):
        c1 = self.ln3(input1, input2)
        combined = c1
        return combined
    
    def relu(self, input1):
        return nn.ReLU(input1)
    
    def output(self, input1):
        c1 = self.ln4(input1)
        return self.tanh(c1)

EB_Model = torch.load('./input/EB_Model.pth')

/opt/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EB_Model.state_dict().keys()

odict_keys(['ln1.weight', 'ln1.bias', 'ln1_2.weight', 'ln2.weight', 'ln2.bias', 'ln2_2.weight', 'ln3.weight', 'ln3.bias', 'ln3_2.weight', 'ln4.weight', 'ln4.bias'])

In [3]:
EB_Model.state_dict()

OrderedDict([('ln1.weight',
              tensor([[[ 0.0067,  0.1000, -0.0935,  ..., -0.0528, -0.0146,  0.0414],
                       [ 0.0848,  0.0932, -0.0068,  ...,  0.0916,  0.0979,  0.0296],
                       [-0.0657,  0.0916,  0.0708,  ..., -0.0927,  0.0057, -0.0397],
                       ...,
                       [ 0.0092,  0.0398, -0.0339,  ..., -0.0919, -0.0209, -0.0207],
                       [-0.0182, -0.0254, -0.0716,  ..., -0.0661, -0.0936, -0.0096],
                       [ 0.0565,  0.0533, -0.0153,  ...,  0.0863, -0.0167, -0.0646]],
              
                      [[-0.0916,  0.0719,  0.0295,  ...,  0.0703,  0.0518,  0.0567],
                       [-0.0536,  0.0241, -0.0333,  ...,  0.0031,  0.0559,  0.0332],
                       [-0.0255,  0.0525, -0.0935,  ...,  0.0842, -0.0583, -0.0299],
                       ...,
                       [ 0.0762,  0.0590, -0.0683,  ..., -0.0113,  0.0474, -0.0480],
                       [-0.0510,  0.0298,  0.0299,

In [234]:
def clearSentence(sentence):
    sentence = re.sub('-KY', '', sentence)
    sentence = re.sub('\*', '', sentence)
    return sentence

def tagging(string):
#     print(string)
    string = string.split('_')
    try:
        re = stockReturns[string[2]][string[0]]
    except:
        try:
            date = str(int(string[0])+1)
            re = stockReturns[string[2]][date]
        except:
            try:
                date = str(int(string[0])+2)
                re = stockReturns[string[2]][date]
            except:
                re = 0
#     print(re)

    if re > 0:
        tag = 1
    elif re < 0:
        tag = 0
    else:
        tag = 2

    return tag


def svo2u(svo):
    (s, v, o) = svo
    svo = []
    for i in (s, v, o):
        svo.append( torch.Tensor(w2v_model.wv.get_vector(i, norm=True)).view(1, 100) )
    
    svo1 = EB_Model.forward1(svo[0], svo[1])
    svo2 = EB_Model.forward2(svo[1], svo[2])
    svo3 = EB_Model.forward3(svo1, svo2)
    
    
    return svo3.detach().numpy()

<>:3: DeprecationWarning: invalid escape sequence \*
<>:3: DeprecationWarning: invalid escape sequence \*
/var/folders/w8/jwxq4stj29506b1pf67xpb000000gq/T/ipykernel_2429/2205918697.py:3: DeprecationWarning: invalid escape sequence \*
  sentence = re.sub('\*', '', sentence)


In [5]:
w2v_model = Word2Vec.load("word2vec2.model")
with open('/Users/arthur/Desktop/Event_Driven_Stock_Prediction_using_Deep_Learning/input2/stockReturns.json') as data_file:
    stockReturns = json.load(data_file)
candidatePoolDF = pd.read_excel('/Users/arthur/Desktop/factor_investment/計算溢酬/CTBCdata/10個因子原始資料.xlsx','125檔', dtype=str,index_col=0,header=1)
candidatePoolDF = candidatePoolDF['名稱'].apply(lambda x: clearSentence(x)).values

# EB_Model = torch.load('./input/EB_Model')

In [6]:
svo2u(['外資', '買', '臺積電'])

/var/folders/w8/jwxq4stj29506b1pf67xpb000000gq/T/ipykernel_2429/89883991.py:36: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_numpy.cpp:178.)
  svo.append( torch.Tensor(w2v_model.wv.get_vector(i, norm=True)).view(1, 100) )


array([[ 0.01994973, -0.03601152,  0.13344684, -0.09484336,  0.0240122 ,
         0.02006958,  0.07168072,  0.05011463, -0.00749791,  0.158745  ,
        -0.0673293 , -0.02467199, -0.06868671,  0.15111825,  0.22681901,
        -0.04604457, -0.12937915, -0.04836076,  0.14658383, -0.04079673,
         0.08158739,  0.10597549, -0.02806954, -0.16710672, -0.00531252,
         0.12138543, -0.08332917, -0.11205601, -0.18436109, -0.05123179,
        -0.1968449 , -0.21279103,  0.02322551,  0.12308085, -0.11800323,
         0.24724   ,  0.04200463, -0.10053404, -0.10278537, -0.04995792,
         0.09467702,  0.15408404, -0.05785674,  0.06047689,  0.06915336,
        -0.10286513, -0.15868872,  0.09959171, -0.09455308,  0.10393374,
        -0.06869697,  0.10745216, -0.12373841, -0.25680602,  0.05043585,
        -0.1462704 , -0.08140966,  0.01833436,  0.05419208, -0.07174428,
        -0.10196389, -0.04187951,  0.20559387,  0.02041125, -0.0930306 ,
        -0.05862888,  0.15592884, -0.01419554, -0.3

In [7]:
print(svo2u(['華爲通訊設備工研院', '擔憂', '華爲設備']))

tensor([[ 1.0900e-01, -4.3144e-02,  6.2867e-02,  5.2415e-02, -5.7773e-02,
          2.6032e-02, -3.4789e-02,  5.1458e-02,  5.6858e-03,  1.2089e-01,
         -8.3006e-03,  1.1420e-01, -5.0184e-02,  9.2442e-02,  6.9269e-02,
          1.0085e-01, -3.3604e-02,  6.4252e-03,  5.9803e-02, -3.5632e-02,
          3.9672e-02,  7.8277e-02, -3.5329e-02, -1.5355e-01,  1.4224e-01,
         -6.4066e-02, -6.7368e-02, -6.6644e-02, -1.0409e-01, -1.1395e-01,
         -8.1207e-02, -2.2525e-01, -1.5871e-01,  6.6456e-02,  2.2521e-03,
          2.0356e-01,  6.4830e-02, -5.3452e-02,  5.3564e-03, -3.4162e-02,
          2.5524e-02,  9.4944e-02, -3.9466e-02, -1.4440e-02,  2.3717e-02,
         -6.5098e-02, -1.1325e-01,  1.0110e-01, -1.2618e-01,  1.1814e-01,
         -1.6643e-02,  6.9431e-02, -6.1626e-02, -1.8855e-01,  3.6616e-02,
         -9.7262e-02, -1.3558e-01, -9.6691e-02,  5.3541e-02, -7.1438e-02,
         -2.3302e-02,  1.2462e-02,  4.6454e-02,  3.7192e-02,  6.7302e-02,
         -9.0540e-02,  6.4079e-02, -1.

# Reference

### 以1家公司30天為假設

input1 = torch.randn(n=30, 1, 30, 100) # 30*monthly news

input2 = torch.randn(n=30, 1, 7, 100) # 30*weekly news

input3 = torch.randn(n=30, 100) # daily news*30

    input4 = torch.randn(100, 1) # company

input5 = torch.randn(n=30) # outcome

    model = ThreeInputsCNN.forward(input1, input2, input3)

# CNN Model

In [151]:
import torch
from torch import nn
import numpy as np
import torch.optim as optim

def to_category(num):
    num[num > 0] = 1
    num[num <= 0] = 0

class ThreeInputsCNN(nn.Module):
    def __init__(self):
        super(ThreeInputsCNN, self).__init__()
        self.conv1 = nn.Sequential(
          nn.Conv2d(1, 100, (3, 100), stride=(1, 1)),
          nn.MaxPool2d((28, 1)),
          nn.Flatten(),
          nn.Flatten()
        )
        self.conv2 = nn.Sequential(
          nn.Conv2d(1, 100, (3, 100), stride=(1, 1)),
          nn.MaxPool2d((5, 1)),
          nn.Flatten(),
          nn.Flatten()
        )
        self.fc = nn.Sequential(
          nn.Linear(3, 1),
          nn.Flatten(),
          nn.Sigmoid(),
          nn.Linear(100, 2),
          nn.Sigmoid()
        )

    def forward(self, input1, input2, input3):
        c1 = self.conv1(input1)
        c2 = self.conv2(input2)
        c3 = input3
    
        self.n = c1.size(0)
        combined = torch.empty(self.n, 100, 3)
        combined[:, :, 0] = c1
        combined[:, :, 1] = c2
        combined[:, :, 2] = c3
        
        out = self.fc(combined)
        return out

In [235]:
def genFeatures(year, month1, month2):
    i = year
    SVO_inputs = pd.DataFrame()
    Label = []

    for j in range(month1, month2): ##### 月 #####-------------------------------------
        print(j, end=', ')
        input_files = [f for f in os.listdir(loc) if f.startswith(str(i)+str(j))]
        input_files = sorted(input_files)

        News = pd.DataFrame()
        for file in input_files: ##### 整月新聞 #####-------------------------------------
            news = pd.read_csv(loc+file)[['News', 'All', 'SVO']]
            News = pd.concat([News, news])

        News = News.reset_index(drop=True) ##### 某月的新聞 #####
        Indiv_News = News.loc[News.All.str.contains(com)].reset_index(drop=True) ##### 某月某公司的新聞 #####

        ##### 加入 「時間」 「漲跌label」 #####
        Indiv_News['Date'] = Indiv_News['News'].apply(lambda x: x[0:8])
        Indiv_News['PriceID'] = Indiv_News['News'].apply(lambda x: x+'_'+str(com))
        Indiv_News['Label'] = Indiv_News['PriceID'].apply(lambda x: tagging(x))
        Indiv_News = Indiv_News[Indiv_News['Label'] != 2].reset_index(drop=True)

        ##### 算出每天的U #####
        for date in sorted( np.unique(Indiv_News['Date']) ): ##### 天 #####-------------------------------------
            Daily_News = Indiv_News[Indiv_News.Date == date].reset_index(drop=True)
            label = Daily_News['Label'][0]
            daily_svo = []

            ##### 算出每天每則的U #####
            for SVO in Daily_News['SVO'].values: ##### 則 #####-------------------------------------
                SVO = ast.literal_eval(SVO)            
                ##### 算出每天每則每組的U #####
                for svo in SVO: ##### 組 #####-------------------------------------
#                             print(svo2u(svo))
                    daily_svo.append(svo2u(svo))
            try:
                warnings.filterwarnings("ignore")
                daily_svo = np.mean(daily_svo, axis=0)[0]
#                     print(daily_svo.shape)
                SVO_inputs = pd.concat([SVO_inputs, pd.DataFrame({'Date': [date], 'U': [daily_svo], 'Label': [label]})]).reset_index(drop=True)
                warnings.filterwarnings("default")
            except:
#                 print(daily_svo)
                pass

#                 print(SVO_inputs)
        print(SVO_inputs.shape[0], end=' | ')
#         if (SVO_inputs.shape[0] < (j*30-40)):
#             print('This company doesn\'t have enough pieces of news. Skip!')
#             Vl, Vm, Vs, y, SVO_inputs = 0, 0, 0, 0, 0
#             return Vl, Vm, Vs, y, SVO_inputs
#             break ###########記得加回來！

    n = SVO_inputs.shape[0] - 30 + 1 ### e.g. 33-30+1=4 (1,30)~(4,33) -> (0,29)~(3,32)
    if n < 2:
        print('y\'s positive rate:', sum(SVO_inputs.loc[:, 'Label'])/len(SVO_inputs.loc[:, 'Label']))
        print('This company doesn\'t have enough pieces of news. Skip!')
        Vl, Vm, Vs, y, SVO_inputs = 0, 0, 0, 0, 0
        return Vl, Vm, Vs, y, SVO_inputs

    Vl = torch.empty(n, 1, 30, 100) # n*1*30*100
    Vm = torch.empty(n, 1, 7, 100) # n*1*7*100
    Vs = torch.empty(n, 100) # n*100         
    y = torch.empty(n) # n

    for i in range(29, SVO_inputs.shape[0]):
        start = i-29
        end = i
        Vl[start, 0, :, :] = torch.Tensor( np.array([i for i in SVO_inputs.loc[start:end, 'U'].values]) )
        Vm[start, 0, :, :] = torch.Tensor( np.array([i for i in SVO_inputs.loc[i-6:i, 'U'].values]) )
        Vs[start, :] = torch.Tensor( SVO_inputs.loc[i, 'U'] )
        y[start] = SVO_inputs.loc[i, 'Label']
        
    return Vl, Vm, Vs, y.long(), SVO_inputs

In [248]:
loc = './input/EB_EventExtraction2/'


for com in candidatePoolDF[1:10]: ##### 公司 #####-------------------------------------
    print('\n'+str(com))
    
    model_CNN = ThreeInputsCNN() ##### 新model #####
    
    for year in range(2019, 2020): ##### 年 #####-------------------------------------
        Vl, Vm, Vs, y, SVO_inputs = genFeatures(year, 1, 7)
        
        if type(Vl) == int: break
        
        print('y\'s positive rate:', sum(y)/len(y))
        ### Training
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model_CNN.parameters(), lr=0.001, momentum=0.9)
        
        
        for epoch in range(1500):

#             running_loss = 0.0

            output = model_CNN.forward(Vl, Vm, Vs)
            loss = criterion(output, y)

            loss.backward()
            optimizer.step()
            if (epoch+1)%100 == 0:
                print(round(float(loss.item()), 3), end=' ')
#             running_loss = 0.0

        print('Finished Training')
        
        Vl, Vm, Vs, y_test, SVO_inputs = genFeatures(year, 7, 9)
        
        if type(Vl) == int: break

        y_predict = model_CNN.forward(Vl, Vm, Vs)
        
        loss = criterion(y_predict, y_test)
        print(round(float(loss.item()), 3), end=' ')
        
        print('y_test\'s positive rate:', sum(y_test)/len(y_test))
        
        thres = 0.5
#         y_cut = (y_predict[:,0] > thres) | (y_predict[:,1] > thres) # cut y value and leave the better result
#         predictions = np.argmax(y_predict.detach().numpy()[y_cut], axis=-1)
        predictions = np.argmax(y_predict.detach().numpy(), axis=-1)
        conf = confusion_matrix(y_test, predictions)
#         print("Test on %d samples" % (len(y_test[y_cut])))
        print(conf)
#         for i in range(clusters):
#         print("Test Label %d Precision, %.2f%%" % (i, conf[i,i] * 100.0 / sum(conf[:,i])))


#                     break
#                 break
#             break
#         break
#     break
#     else: ##### test #####
#         for j in range(1, 2):
#             break


鴻海
1, 25 | 2, 38 | 3, 62 | 4, 87 | 5, 110 | 6, 135 | y's positive rate: tensor(0.5377)
0.707 0.709 0.691 0.698 0.69 0.7 0.691 0.693 0.695 0.694 0.694 0.693 0.693 0.693 0.693 Finished Training
7, 25 | 8, 50 | 0.693 y_test's positive rate: tensor(0.6190)
[[ 8  0]
 [13  0]]

台塑化
1, 15 | 2, 24 | 3, 33 | 4, 42 | 5, 58 | 6, 68 | y's positive rate: tensor(0.5385)
0.698 0.738 0.693 0.714 0.691 0.695 0.695 0.694 0.694 0.693 0.693 0.693 0.693 0.693 0.693 Finished Training
7, 10 | 8, 21 | y's positive rate: 0.38095238095238093
This company doesn't have enough pieces of news. Skip!

中華電
1, 11 | 2, 17 | 3, 25 | 4, 35 | 5, 44 | 6, 55 | y's positive rate: tensor(0.5769)
0.691 0.693 0.694 0.695 0.694 0.693 0.693 0.693 0.693 0.693 0.693 0.693 0.693 0.693 0.693 Finished Training
7, 13 | 8, 29 | y's positive rate: 0.5172413793103449
This company doesn't have enough pieces of news. Skip!

聯發科
1, 18 | 2, 31 | 3, 52 | 4, 72 | 5, 92 | 6, 108 | y's positive rate: tensor(0.5570)
0.689 0.702 0.694 0.692 0.69 0

In [10]:
# svo2u(['外資', '買', '臺積電'])
# svo2u(['買', '買', '臺積電'])
# svo2u(['華爲通訊設備工研院', '買', '臺積電'])

In [11]:
# EB_Model.forward1(torch.Tensor(w2v_model.wv.get_vector('外資', norm=True)), torch.Tensor(w2v_model.wv.get_vector('買', norm=True)))
#                         w2v_model.wv.get_vector(i, norm=True)

In [12]:
# EB_Model.forward1(torch.Tensor(w2v_model.wv.get_vector('買', norm=True)), torch.Tensor(w2v_model.wv.get_vector('買', norm=True)))

In [222]:
test = pd.DataFrame({'PriceID': ['20190401_888_鴻海', '20190402_888_鴻海', '20190403_888_鴻海']})
test

,PriceID
0,20190401_888_鴻海
1,20190402_888_鴻海
2,20190403_888_鴻海


In [223]:
test['Label'] = test['PriceID'].apply(lambda x: tagging(x))
test

,PriceID,Label
0,20190401_888_鴻海,1
1,20190402_888_鴻海,1
2,20190403_888_鴻海,0


In [224]:
sum(test.loc[:, 'Label'])/len(test.loc[:, 'Label'])

0.6666666666666666